In [ ]:

# ==============================================================================
# 2. THE EXPERIMENT
# ==============================================================================
def run_optimization_experiment():
    print("Running Single Receptor Optimization...")
    
    # A. CONFIGURATION
    # ----------------
    # Single unit 'A', Homopentamer 'AAAAA'
    n_units = 1 
    k_sub = 5
    receptor_indices = torch.zeros(1, k_sub, dtype=torch.long) # [[0, 0, 0, 0, 0]]
    
    # Initialize
    env = SimpleNormalEnvironment()
    physics = MWCReceptorLayer(n_units, k_sub)
    loss_fn = InformationLoss()
    
    optimizer = optim.Adam(list(env.parameters()) + list(physics.parameters()), lr=0.05)
    
    # B. SNAPSHOTS (For Plotting)
    # ---------------------------
    # We want to see the curve BEFORE and AFTER.
    
    def get_response_curve(env_model, phys_model):
        """Helper to compute the deterministic dose-response curve."""
        c_range = torch.linspace(0, 10, 100)
        
        # Use MEAN energies (no noise) for the curve plotting
        mean_energies = env_model.interaction_mu # (1, 1, 2)
        # Expand energies to match c_range batch
        energies_batch = mean_energies.expand(100, -1, -1)
        
        with torch.no_grad():
            activity = phys_model(energies_batch, c_range, receptor_indices)
        return c_range.numpy(), activity.squeeze().numpy()

    # Capture Initial State
    c_grid, p_open_initial = get_response_curve(env, physics)
    
    # C. TRAINING LOOP
    # ----------------
    print("Optimizing...", end="")
    history = []
    for i in range(500):
        optimizer.zero_grad()
        
        # Sample & Simulate
        energies, concs, _ = env.sample_batch(batch_size=512)
        activity = physics(energies, concs, receptor_indices)
        
        # Maximize Entropy (Minimize negative entropy)
        # We ignore covariance since we have only 1 receptor
        loss, stats = loss_fn(activity)
        
        loss.backward()
        optimizer.step()
        
        history.append(stats['entropy'].item())
        if i % 100 == 0: print(f".", end="")
            
    print(" Done!")
    
    # Capture Final State
    _, p_open_final = get_response_curve(env, physics)

    # ==============================================================================
    # 3. PLOTTING
    # ==============================================================================
    # Define the Ideal Input Distribution for visualization
    # Normal(5, 1)
    x = np.linspace(0, 10, 200)
    pdf = (1 / (np.sqrt(2 * np.pi))) * np.exp(-0.5 * (x - 5)**2)
    # Normalize PDF to fit in plot roughly (just for visual shape)
    pdf_scaled = pdf / pdf.max() 
    
    plt.figure(figsize=(10, 6))
    
    # 1. Plot Concentration Distribution (The Target)
    plt.fill_between(x, pdf_scaled, color='gray', alpha=0.2, label='Ligand Conc. P(c)')
    
    # 2. Plot Initial Receptor Response
    plt.plot(c_grid, p_open_initial, 'r--', linewidth=2, label='Receptor (Before)')
    
    # 3. Plot Optimized Receptor Response
    plt.plot(c_grid, p_open_final, 'g-', linewidth=3, label='Receptor (After)')
    
    plt.title("Optimization of Single Receptor Response")
    plt.xlabel("Concentration (c)")
    plt.ylabel("Probability Open / Scaled Density")
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Save
    output_dir = "unit_test/plots"
    os.makedirs(output_dir, exist_ok=True)
    save_path = os.path.join(output_dir, "single_receptor_opt.png")
    plt.savefig(save_path)
    print(f"Plot saved to {save_path}")

if __name__ == "__main__":
    run_optimization_experiment()